In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

traindatagen = ImageDataGenerator(
    # augmentation
        rotation_range=45,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

valdatagen = ImageDataGenerator()

Using TensorFlow backend.


In [2]:
batch_size = 32
from sklearn.utils import shuffle

# output: directory iterator
train_generator = traindatagen.flow_from_directory(
    directory='train', 
    shuffle = True,
        target_size=(299,299),  # all images will be resized to 299x299
        batch_size=batch_size,
        class_mode='categorical')  
# since we use binary_crossentropy loss, we need binary labels

Found 7244 images belonging to 2 classes.


In [3]:
test_generator = valdatagen.flow_from_directory(
    directory='validation', 
        shuffle = False,
        target_size=(299,299),  # all images will be resized to 299x299
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

Found 1878 images belonging to 2 classes.


In [4]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras import optimizers
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

In [5]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)
# whether to include the fully-connected layer at the top of the network.

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

In [6]:
# let's add a fully-connected layer
# first arg is the dim of output
x = Dense(16, activation='relu')(x)
# and a logistic layer -- let's say we have 2 classes
predictions = Dense(2, activation='softmax')(x)
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

nb_epoch = 100

## Train the model 
TrainingLogs = model.fit_generator(
    train_generator,
    steps_per_epoch=230,
    validation_data=test_generator,
    validation_steps=60,
    epochs=nb_epoch,
    callbacks=[early_stopping],
    verbose=1,
    pickle_safe=False
)

Epoch 1/100
230/230 [==============================] - 156s - loss: 0.3896 - acc: 0.8232 - val_loss: 0.8042 - val_acc: 0.6450

In [8]:
heldout = valdatagen.flow_from_directory(directory='test', 
        target_size=(299,299),  # all images will be resized to 299x299
        batch_size=batch_size,
        class_mode='categorical') 

Found 2294 images belonging to 2 classes.


In [9]:
res = model.evaluate_generator(heldout, steps=10, max_q_size=10, workers=1, pickle_safe=False)

# res = model.evaluate_generator(heldout, steps=3, max_q_size=10, workers=1, pickle_safe=False)
print(model.metrics_names, '\n',res)

['loss', 'acc'] 
 [0.54665203094482417, 0.77500000000000002]


In [10]:
res2 = model.evaluate_generator(heldout, steps=72, max_q_size=10, workers=1, pickle_safe=False)
print(model.metrics_names, '\n',res2)

['loss', 'acc'] 
 [0.49528459273116321, 0.7916303402772652]


In [13]:
predictions = model.predict_generator(heldout, steps=72)
predicted_classes = np.argmax(predictions, axis=1)

true_classes = heldout.classes
class_labels = list(heldout.class_indices.keys())   

In [18]:
from sklearn.metrics import classification_report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)  

             precision    recall  f1-score   support

NonViolence       0.47      0.33      0.39      1037
   Violence       0.55      0.69      0.61      1257

avg / total       0.51      0.53      0.51      2294

